# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

# ADD: add "eos" in each french sentences
french_sentences = [sen+" <EOS>" for sen in french_sentences]

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <EOS>
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre . <EOS>


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [3]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

2099156 French words.
356 unique French words.
10 Most common words in the French dataset:
"est" "<EOS>" "." "," "en" "il" "les" "mais" "et" "la"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [4]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [5]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(x)
    x_tokenized = tokenizer.texts_to_sequences(x)
    return x_tokenized, tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'my': 12, 'won': 16, 'of': 14, 'this': 18, 'quick': 2, 'lexicography': 15, 'sentence': 21, 'by': 10, 'fox': 5, 'short': 20, 'brown': 4, 'the': 1, 'a': 3, 'prize': 17, 'lazy': 8, 'study': 13, 'jove': 11, 'dog': 9, 'over': 7, 'is': 19, 'jumps': 6}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [6]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [7]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding="post")
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [8]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


In [9]:
preproc_english_sentences.shape, preproc_french_sentences.shape

((137861, 15), (137861, 22, 1))

In [10]:
len(english_tokenizer.word_index), len(french_tokenizer.word_index)

(199, 345)

In [11]:
french_tokenizer.word_index["<eos>"]

2

## Constants
Note: this section require changes when the dataset is changed.

In [12]:
ENG_VOCAB_SIZE = 200 #len(eng_vocab) + 1
FRE_VOCAB_SIZE = 347 #len(french_vocab) + 2
ENG_SEQ_LEN = 15
FRE_SEQ_LEN = 22
TGT_S_INDEX = 346 #len(french_vocab)
TGT_E_INDEX = 2

BATCH_SIZE = 128

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [13]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [14]:
from keras.layers import GRU, LSTM, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [15]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    m_in = Input(shape=input_shape[1:])
    x = GRU(128, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)(m_in)
    m_out = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(x)
    model = Model(m_in, m_out)
    
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

In [17]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    ENG_VOCAB_SIZE,
    FRE_VOCAB_SIZE
)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=2, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/2
110288/110288 [==============================] - 116s - loss: 1.8886 - acc: 0.5344 - val_loss: 2.0964 - val_acc: 0.5105
Epoch 2/2
110288/110288 [==============================] - 120s - loss: 1.6728 - acc: 0.5560 - val_loss: 2.0331 - val_acc: 0.5238
new fruit est la agréable en en mais il est est est en <eos> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [18]:
from keras.layers.embeddings import Embedding


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    m_in = Input(shape=input_shape[1:])
    x = Embedding(english_vocab_size, 64)(m_in)
    x = GRU(128, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)(x)
    m_out = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(x)
    model = Model(m_in, m_out)
    
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

In [19]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    ENG_VOCAB_SIZE,
    FRE_VOCAB_SIZE)
rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=2, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/2
110288/110288 [==============================] - 140s - loss: 0.8773 - acc: 0.7473 - val_loss: 0.4869 - val_acc: 0.8408
Epoch 2/2
110288/110288 [==============================] - 134s - loss: 0.5995 - acc: 0.8051 - val_loss: 0.4740 - val_acc: 0.8458
new jersey est parfois calme en l'automne et il est neigeux en en <eos> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [20]:
from keras.layers import Bidirectional, Dropout


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    m_in = Input(shape=input_shape[1:])
    x = Bidirectional(GRU(128, return_sequences=True))(m_in)
    x = Dropout(0.5)(x)
    m_out = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(x)
    model = Model(m_in, m_out)
    
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_bd_model(bd_model)

In [21]:
# TODO: Train and Print prediction(s)
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    ENG_VOCAB_SIZE,
    FRE_VOCAB_SIZE)
rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=2, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/2
110288/110288 [==============================] - 214s - loss: 1.3029 - acc: 0.6270 - val_loss: 1.0249 - val_acc: 0.6751
Epoch 2/2
110288/110288 [==============================] - 194s - loss: 1.1061 - acc: 0.6594 - val_loss: 0.9418 - val_acc: 0.6923
new jersey est parfois calme en l' et il il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 4: Embedding-Bidirectional-EncoderDecoder (IMPLEMENTATION)
![RNN](images/bidirectional.png)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [22]:
from keras.models import Sequential
from keras.layers import RepeatVector

def model_embed_bd(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    
    '''Encoder'''
    model.add(Embedding(english_vocab_size, 64, input_length=input_shape[1]))
    model.add(Bidirectional(GRU(128, return_sequences=False)))
    model.add(Dropout(0.5))
    
    '''Decoder'''
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size, activation="softmax")))
    
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model

tests.test_model_final(model_embed_bd)


print('Model 4 Loaded')

Model 4 Loaded


In [23]:
# TODO: Train and Print prediction(s)
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
embed_bd_model = model_embed_bd(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    ENG_VOCAB_SIZE,
    FRE_VOCAB_SIZE)
embed_bd_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=5, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_bd_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/5
110288/110288 [==============================] - 401s - loss: 1.1514 - acc: 0.6862 - val_loss: 0.6242 - val_acc: 0.8127
Epoch 2/5
110288/110288 [==============================] - 392s - loss: 1.0303 - acc: 0.7191 - val_loss: 1.7124 - val_acc: 0.5619
Epoch 3/5
110288/110288 [==============================] - 387s - loss: 1.7538 - acc: 0.5544 - val_loss: 1.5045 - val_acc: 0.6025
Epoch 4/5
110288/110288 [==============================] - 392s - loss: 1.8405 - acc: 0.5400 - val_loss: 1.5736 - val_acc: 0.5838
Epoch 5/5
110288/110288 [==============================] - 521s - loss: 1.7451 - acc: 0.5550 - val_loss: 1.5469 - val_acc: 0.5897
new jersey est parfois au en mois mais il est est est en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Evaluate the embedding bidirectional model

In [58]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = embed_bd_model

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    y_id_to_word[TGT_S_INDEX] = "<S>"
    y_id_to_word[TGT_E_INDEX] = "<EOS>"

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0],x[0]])
    sentences = pad_sequences(sentences, maxlen=y.shape[-2], padding="post")
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[x[0]] for x in y[0]]))
    
#     print('Sample 1:')
#     print(' '.join([y_id_to_word[x] for x in predictions[0]]))
#     print('Il a vu un vieux camion jaune')
#     print('Sample 2:')
#     print(' '.join([y_id_to_word[x] for x in predictions[1]]))
#     print(' '.join([y_id_to_word[x] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il a vu vieux camion jaune <EOS> <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois au en mois mais il est est est en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 5: Encoder-Decoder With Attention (IMPLEMENTATION)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In order to performance well for long sentences, attention mechanism is used.

Create an encoder-decoder model in the cell below.

![seq2seq](images/seq2seq.png)

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow.python.layers.core import Dense

In [35]:
X_vocab_size = ENG_VOCAB_SIZE
Y_vocab_size = FRE_VOCAB_SIZE

g_nmt = tf.Graph()

with g_nmt.as_default():
    '''variables'''
    X = tf.placeholder(tf.int32, [None, None])
    Y = tf.placeholder(tf.int32, [None, None])
    D = tf.placeholder(tf.int32, [None, None])
    keep_prob = tf.placeholder(tf.float32)
    #X_seq_len = tf.placeholder(tf.int32, [None])
    #Y_seq_len = tf.placeholder(tf.int32, [None])
    X_seq_len = tf.constant([ENG_SEQ_LEN for i in range(BATCH_SIZE)])
    Y_seq_len = tf.constant([FRE_SEQ_LEN for i in range(BATCH_SIZE)])

    '''encoder''' 
    encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(128)
    encoder_cell = tf.nn.rnn_cell.DropoutWrapper(encoder_cell, keep_prob, keep_prob, keep_prob)
    encoder_out, encoder_state = tf.nn.dynamic_rnn(
        cell = encoder_cell, 
        inputs = tf.contrib.layers.embed_sequence(X, X_vocab_size, 64),
        sequence_length = X_seq_len,
        dtype = tf.float32)
    
    '''decoder cell'''
    decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(128)
    decoder_cell = tf.nn.rnn_cell.DropoutWrapper(decoder_cell, keep_prob, keep_prob, keep_prob)

    '''attention'''
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        num_units = 128, 
        memory = encoder_out,
        memory_sequence_length = X_seq_len)

    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
        cell = decoder_cell,
        attention_mechanism = attention_mechanism,
        attention_layer_size = 128)

    '''decoder'''
    projection_layer = Dense(Y_vocab_size)

    embedding_decoder = tf.get_variable("embedding_decoder", [Y_vocab_size, 64])
    training_helper = tf.contrib.seq2seq.TrainingHelper(
        inputs = tf.nn.embedding_lookup(
            embedding_decoder, D),
        sequence_length = Y_seq_len,
        time_major = False)
    training_decoder = tf.contrib.seq2seq.BasicDecoder(
        cell = decoder_cell,
        helper = training_helper, 
        initial_state = decoder_cell.zero_state(BATCH_SIZE, tf.float32).clone(cell_state=encoder_state),
        output_layer = projection_layer)
    training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
        decoder = training_decoder)
    training_logits = training_decoder_output.rnn_output

    '''loss'''
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=Y, logits=training_logits)
    train_loss = (tf.reduce_sum(crossent) /
            BATCH_SIZE)
    '''backward'''
    params = tf.trainable_variables()
    gradients = tf.gradients(train_loss, params)
    clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
    train_op = tf.train.AdamOptimizer().apply_gradients(zip(clipped_gradients, params))

    '''infer'''
    tgt_sos_id = TGT_S_INDEX
    tgt_eos_id = TGT_E_INDEX

    infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        embedding_decoder,
        tf.fill([BATCH_SIZE], tgt_sos_id), tgt_eos_id) # try int, and place holder

    # Decoder
    decoder_infer = tf.contrib.seq2seq.BasicDecoder(
        decoder_cell, infer_helper,
        initial_state = decoder_cell.zero_state(BATCH_SIZE, tf.float32).clone(cell_state=encoder_state),
        output_layer=projection_layer)

    # Dynamic decoding
    outputs_infer = tf.contrib.seq2seq.dynamic_decode(
        decoder_infer)

    
    translations = outputs_infer[0].sample_id

In [36]:
from sklearn.cross_validation import train_test_split

X_all = preproc_english_sentences
Y_all = preproc_french_sentences.reshape(preproc_french_sentences.shape[:2])
D_all = np.hstack([np.ones([Y_all.shape[0], 1])*TGT_S_INDEX, Y_all[:, :-1]]) # decoder input

X_train, X_test, Y_train, Y_test, D_train, D_test = train_test_split(X_all, Y_all, D_all)


/Users/jhsiao/anaconda/envs/py3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [40]:
'''training'''
import time

num_of_dp = X_train.shape[0]

DISPLAY_STEP = 250

EPOCH = 5

iters_each_epoch = X_train.shape[0] // BATCH_SIZE


with tf.Session(graph=g_nmt) as sess:
    sess.run(tf.global_variables_initializer())
    t0 = time.time()
    
    for i in range(EPOCH):
        print("\n### EPOCH %d\n"%i)
        for j in range(iters_each_epoch):
            start_id = BATCH_SIZE * j
            end_id = BATCH_SIZE * (j+1)
            _, loss = sess.run(
                [train_op, train_loss],
                {
                    X:X_train[start_id:end_id],
                    Y:Y_train[start_id:end_id],
                    D:D_train[start_id:end_id],
                    keep_prob:0.5
                }
            )
            if j%DISPLAY_STEP == 0:
                print("Epoch: %d, Process: %d/%d, Loss: %f"%(i, start_id, num_of_dp, loss))
                t1 = time.time()
                print("Time use: %.2f"%(t1-t0))
                t0 = t1
        
        # eval
        val_loss = sess.run(
                [train_loss],
                {
                    X:X_test[:BATCH_SIZE],
                    Y:Y_test[:BATCH_SIZE],
                    D:D_test[:BATCH_SIZE],
                    keep_prob:1.0
                }
            )
        print("val loss:", val_loss)
        
        # save
        saver = tf.train.Saver()
        saver.save(sess, "./nmt_sess")
        print("model trained and saved")



### EPOCH 0

Epoch: 0, Process: 0/103395, Loss: 128.919464
Time use: 0.45
Epoch: 0, Process: 32000/103395, Loss: 27.209320
Time use: 53.60
Epoch: 0, Process: 64000/103395, Loss: 20.111822
Time use: 56.93
Epoch: 0, Process: 96000/103395, Loss: 16.422218
Time use: 56.63
val loss: [13.979229]
model trained and saved

### EPOCH 1

Epoch: 1, Process: 0/103395, Loss: 15.448201
Time use: 14.67
Epoch: 1, Process: 32000/103395, Loss: 12.732707
Time use: 60.25
Epoch: 1, Process: 64000/103395, Loss: 8.943460
Time use: 64.34
Epoch: 1, Process: 96000/103395, Loss: 8.062677
Time use: 61.92
val loss: [5.942421]
model trained and saved

### EPOCH 2

Epoch: 2, Process: 0/103395, Loss: 7.552980
Time use: 16.29
Epoch: 2, Process: 32000/103395, Loss: 6.401490
Time use: 56.56
Epoch: 2, Process: 64000/103395, Loss: 5.238935
Time use: 56.34
Epoch: 2, Process: 96000/103395, Loss: 5.058176
Time use: 61.36
val loss: [3.8008645]
model trained and saved

### EPOCH 3

Epoch: 3, Process: 0/103395, Loss: 4.953457
T

### Model 6: Attention Is All You Need (IMPLEMENTATION)
The transformer, outperformed the state-of-the-art one in machine translation with only attention, no CNNs, no RNNs. How cool it is! 
![Attention](images/transformer.png)

The implementation used for this model is here: https://github.com/Kyubyong/transformer

In [41]:
import sys
sys.path.append("./transformer")
from transformer.train import Graph

In [42]:
from transformer.hyperparams import Hyperparams as hp

In [43]:
# load the graph of transformer
g = Graph(ENG_VOCAB_SIZE, FRE_VOCAB_SIZE)

In [44]:
X_all = np.hstack([X_all, np.zeros([X_all.shape[0], FRE_SEQ_LEN-ENG_SEQ_LEN])]) #pad it to len=22

In [45]:
X_train, X_test, Y_train, Y_test, D_train, D_test = train_test_split(X_all, Y_all, D_all)

In [47]:
'''training the transformer'''
import time

num_of_dp = X_train.shape[0]

DISPLAY_STEP = 200

EPOCH = 15

iters_each_epoch = X_train.shape[0] // BATCH_SIZE


with tf.Session(graph=g.graph) as sess:
    sess.run(tf.global_variables_initializer())
    t0 = time.time()
    
    for i in range(EPOCH):
        print("\n### EPOCH %d\n"%i)
        for j in range(iters_each_epoch):
            start_id = BATCH_SIZE * j
            end_id = BATCH_SIZE * (j+1)
            _, loss, acc = sess.run(
                [g.train_op, g.mean_loss, g.acc],
                {
                    g.x:X_train[start_id:end_id],
                    g.y:Y_train[start_id:end_id],
                }
            )
            if j%DISPLAY_STEP == 0:
                print("Epoch: %d, Process: %d/%d, Loss: %f, Acc: %f"%(i, start_id, num_of_dp, loss, acc))
                t1 = time.time()
                print("Time use: %.2f"%(t1-t0))
                t0 = t1
        
        # eval
        val_loss = sess.run(
                [g.mean_loss],
                {
                    g.x:X_test[:BATCH_SIZE],
                    g.y:Y_test[:BATCH_SIZE],
                }
            )
        print("val loss:", val_loss)
        
        # save
        gs = sess.run(g.global_step) 
        tf.train.Saver().save(sess, hp.logdir + '/model_epoch_%02d_gs_%d' % (i, gs))
        print("model trained and saved")



### EPOCH 0

Epoch: 0, Process: 0/103395, Loss: 6.118505, Acc: 0.000590
Time use: 1.40
Epoch: 0, Process: 25600/103395, Loss: 1.909913, Acc: 0.606308
Time use: 185.65
Epoch: 0, Process: 51200/103395, Loss: 1.776961, Acc: 0.658551
Time use: 192.03
Epoch: 0, Process: 76800/103395, Loss: 1.771750, Acc: 0.657760
Time use: 186.44
Epoch: 0, Process: 102400/103395, Loss: 1.727365, Acc: 0.676968
Time use: 175.25
val loss: [1.7240031]
model trained and saved

### EPOCH 1

Epoch: 1, Process: 0/103395, Loss: 1.723728, Acc: 0.688496
Time use: 18.04
Epoch: 1, Process: 25600/103395, Loss: 1.703067, Acc: 0.683995
Time use: 190.54
Epoch: 1, Process: 51200/103395, Loss: 1.690135, Acc: 0.698931
Time use: 184.44
Epoch: 1, Process: 76800/103395, Loss: 1.668147, Acc: 0.700398
Time use: 184.58
Epoch: 1, Process: 102400/103395, Loss: 1.625366, Acc: 0.708455
Time use: 181.23
val loss: [1.6264958]
model trained and saved

### EPOCH 2

Epoch: 2, Process: 0/103395, Loss: 1.632003, Acc: 0.717404
Time use: 17.86


## Prediction (IMPLEMENTATION)

In [48]:
# build the inference graph
transformer_infer = Graph(ENG_VOCAB_SIZE, FRE_VOCAB_SIZE, False)

In [56]:
def transformer_predict(x, batch_size):
    # TODO: inference using transformer
    
    # pad to 22
    x = np.hstack([x, np.zeros([x.shape[0], FRE_SEQ_LEN-ENG_SEQ_LEN])]) #pad it to len=22
    
     ### Autoregressive inference
    preds = np.zeros((batch_size, FRE_SEQ_LEN), np.int32)
    
    with tf.Session(graph=g.graph) as sess:
        #sess.run(tf.global_variables_initializer())
        
        load_path = tf.train.latest_checkpoint(hp.logdir)
        loader = tf.train.import_meta_graph(load_path + '.meta')
        loader.restore(sess, load_path)
        print("restored")
        
        for j in range(FRE_SEQ_LEN):
            #_preds = sess.run(transformer_infer.preds, {transformer_infer.x: x, transformer_infer.y: preds})
            _preds = sess.run(g.preds, {g.x: x, g.y: preds})# TODO: run that
            preds[:, j] = _preds[:, j]

    return preds

### Evaluate Transformer

In [57]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    y_id_to_word[TGT_S_INDEX] = "<S>"
    y_id_to_word[TGT_E_INDEX] = "<EOS>"

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = transformer_predict(sentences, len(sentences))

#     print('Sample 1:')
#     print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
#     print('Il a vu un vieux camion jaune')
#     print('Sample 2:')
#     print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
#     print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))
    
    print('Sample 1:')
    print(' '.join([y_id_to_word[x] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[x] for x in predictions[1]]))
    print(' '.join([y_id_to_word[x[0]] for x in y[0]]))



final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

INFO:tensorflow:Restoring parameters from logdir/model_epoch_14_gs_12105
restored
Sample 1:
il a vu un camion jaune brillant <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> voiture jaune <EOS> <EOS> est <EOS> <EOS> <EOS> <EOS>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois agréable en avril et il est neigeux en septembre <EOS> en septembre <EOS> en février <EOS> en juillet
new jersey est parfois calme pendant l' automne et il est neigeux en avril <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.